In [1]:
import networkx as nx
import random

In [2]:
N = 500
g = nx.random_regular_graph(15, N, 1234)

In [3]:
nx.average_neighbor_degree(g)[0]

15.0

In [4]:
attr = {i:(1.0 if i%2==0 else 0.0) for i in range(N)}
nx.set_node_attributes(g, attr, "h")
g.nodes[6]

{'h': 1.0}

In [5]:
def calc_homophily(g):
    total = 0
    count = 0
    for e in g.edges:
        total += 1
        if g.nodes[e[0]]['h'] == g.nodes[e[1]]['h']:
            count += 1
    return(count, total)
calc_homophily(g)

(1890, 3750)

In [6]:
# swap attributes of node i and j
def swap_h(g, i, j):
    temp = g.nodes[i]['h']
    g.nodes[i]['h'] = g.nodes[j]['h']
    g.nodes[j]['h'] = temp
swap_h(g, 0, 1)

In [11]:
def iterate_swap(g):
    p,ne = calc_homophily(g)
    es = list(g.edges)
    for t in range(2000):
        print(p)
        r = random.randrange(0,len(es))
        n1,n2 = es[r]
        if g.nodes[n1]['h'] == g.nodes[n2]['h']:
            continue
        swap_h(g,n1,n2)
        p2,ne = calc_homophily(g)
        if p >= p2:
            swap_h(g,n1,n2)  # reject the swap
            #print("rejected")
        else:
            #print("accepted")
            p = p2
    
#iterate_swap(g)
p,ne = calc_homophily(g)
(p,ne)

(2338, 3750)